# driving distance
https://towardsdatascience.com/driving-distance-between-two-or-more-places-in-python-89779d691def



In [1]:
import requests
import json
import pandas as pd

In [2]:
lat_1, lon_1  =  45.650200,	 13.767620  # Trieste Piazza Unità
lat_2, lon_2  =  45.656938,  13.829219   # campus Padriciano
# call the OSMR API
r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat_1};{lon_2},{lat_2}?overview=full&geometries=geojson""")
# then you load the response using the json libray
# by default you get only one alternative so you access 0-th element of the `routes`
routes = json.loads(r.content)
route_1 = routes.get("routes")[0]


In [3]:
# import os
# path = os.getcwd()
# print(path)
# # with os.scandir(path) as listOfEntries:
# #     print(f'Listing files and dirs in {path}')# print all entries that are files

# #     for entry in listOfEntries:
# #         if entry.is_file():
# #             print(entry.name)
# #         if not entry.is_file():
# #             print('>>> DIR: ', entry.name)

 

In [4]:
filename = r'orig-dest.xlsx'
orig_cols = ['n', 'OR', 'orig', 'Lat', 'Long']
dforig = pd.read_excel(filename, sheet_name = "ORIG", usecols = orig_cols)
dforig.set_index('n', inplace = True)
dforig.dropna(axis = 0, how = 'any', subset = ['Lat'], inplace = True)

dest_cols = ['n', 'DD', 'dest', 'Lat', 'Long']
dfdest = pd.read_excel(filename, sheet_name = "DEST", usecols = dest_cols)
dfdest.set_index('n', inplace = True)
dfdest.dropna(axis = 0, how = 'any', subset = ['Lat'], inplace = True)




In [5]:
dfdest

,DD,dest,Lat,Long
n,,,,
1,TS_P,Campus Padriciano,45.656938,13.829219
2,TS_B,Campus Basovizza,45.644505,13.846497
3,UD_FI,"Friuli Innovazione, VIA JACOPO LINUSSIO 51, Ud...",46.014887,13.267489
4,UD_C,"Confindustria Udine, Dei Torriani 2, Udine, UD...",46.063948,13.231134
5,UD_S,NaN,46.085519,13.206124
6,UD_A,"Via Jacopo Linussio 1, Amaro (UD)",46.370194,13.077139
7,LEF,Zona Industriale Ponterosso PN,45.944881,12.880890
8,PN,Via Villanova di Sotto 16 - 33170 Pordenone (P...,45.932598,12.675420


In [6]:
all_routes = pd.DataFrame(columns= ['o', 'd', 'dist', 'dur', 'geom']) #empty


for i, orig in dforig.iterrows():
    lat_1, long_1  =  orig['Lat'], orig['Long'] 
    siglaO = orig["OR"]

    for j, dest in dfdest.iterrows():
        lat_2, long_2  =  dest['Lat'], dest['Long'] 
        siglaD = dest["DD"]
        
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{long_1},{lat_1};{long_2},{lat_2}?overview=full&geometries=geojson""")
        routes = json.loads(r.content)
        route_1 = routes.get("routes")[0] #select only forst route for each jorney
        dist =  int(route_1.get("distance")/10)/100
        dur =   route_1.get("duration") #seconds
        geom =  route_1.get("geometry")['coordinates']

        
        all_routes = all_routes.append({'o': siglaO, 'd': siglaD, 
                                        'dist': dist,
                                        'dur':dur, 
                                        'geom':geom}, ignore_index=True)
        
        print(f'Origine {i} {siglaO} Lat {lat_1} Long {long_1}')
        print(f'Destinazione {j} {siglaD} Lat {lat_2} Long {long_2}')
        print(dist, dur)
        print('\n\n')
        

Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 1 TS_P Lat 45.656938 Long 13.829219
10.56 747



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 2 TS_B Lat 45.644505 Long 13.846497
10.14 814.9



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 3 UD_FI Lat 46.014887 Long 13.267489
71.04 3225.1



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 4 UD_C Lat 46.063948 Long 13.231134
73.27 3299.5



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 5 UD_S Lat 46.085519 Long 13.206124
76.67 3435.6



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 6 UD_A Lat 46.370194 Long 13.077139
112.84 4551.6



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 7 LEF Lat 45.944881 Long 12.88089
91.31 5101.8



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 8 PN Lat 45.932598 Long 12.67542
111.88 4581.7



Origine 2 TS2 Lat 45.649826 Long 13.777175
Destinazione 1 TS_P Lat 45.656938 Long 13.829219
9.89 727.4



Origine 2 TS2 Lat 45.649826 Long 13.777175
Destinazione 2 TS

In [7]:
all_routes.head(5)


,o,d,dist,dur,geom
0,TS1,TS_P,10.56,747,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
1,TS1,TS_B,10.14,814.9,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
2,TS1,UD_FI,71.04,3225.1,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
3,TS1,UD_C,73.27,3299.5,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
4,TS1,UD_S,76.67,3435.6,"[[13.767103, 45.650613], [13.76746, 45.650831]..."


In [22]:
#

0 0 [13.767103, 45.650613]
0 1 [13.76746, 45.650831]
0 2 [13.767543, 45.650881]
0 3 [13.767895, 45.651096]
0 4 [13.768403, 45.651387]
0 5 [13.768473, 45.651396]
0 6 [13.768544, 45.6514]
0 7 [13.768622, 45.6514]
0 8 [13.7687, 45.651391]
0 9 [13.768773, 45.651378]
0 10 [13.768881, 45.651327]
0 11 [13.768978, 45.651282]
0 12 [13.769352, 45.651077]
0 13 [13.769943, 45.650746]
0 14 [13.770107, 45.650652]
0 15 [13.770146, 45.650631]
0 16 [13.770207, 45.6506]
0 17 [13.77035, 45.650552]
0 18 [13.770648, 45.650473]
0 19 [13.771113, 45.650346]
0 20 [13.771196, 45.650323]
0 21 [13.771265, 45.650305]
0 22 [13.772004, 45.650108]
0 23 [13.77212, 45.650077]
0 24 [13.772238, 45.650045]
0 25 [13.772484, 45.649979]
0 26 [13.772567, 45.649957]
0 27 [13.773051, 45.649834]
0 28 [13.773325, 45.649779]
0 29 [13.773403, 45.649762]
0 30 [13.773849, 45.649669]
0 31 [13.773914, 45.649657]
0 32 [13.774018, 45.649639]
0 33 [13.774774, 45.649617]
0 34 [13.775488, 45.649611]
0 35 [13.776101, 45.649607]
0 36 [13.7762

In [15]:
all_routes.to_csv('all_routes.csv', index = False)


In [17]:
all_routes = pd.read_csv('all_routes.csv', 
                    usecols = ['o', 'd', 'dist', 'dur', 'geom'], 
                    dtype = {'dist':float, 'dur':float},
                    converters={'geom': eval})
    #la lista 'geom' viene scritta nel file csv sotto forma di stringa
    #converters={'geom': eval} la converte in pandas series
                    


 

,o,d,dist,dur,geom
0,TS1,TS_P,10.56,747.0,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
1,TS1,TS_B,10.14,814.9,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
2,TS1,UD_FI,71.04,3225.1,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
3,TS1,UD_C,73.27,3299.5,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
4,TS1,UD_S,76.67,3435.6,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
...,...,...,...,...,...
339,SL2,UD_C,78.14,3298.4,"[[13.873725, 45.709296], [13.873744, 45.709304..."
340,SL2,UD_S,81.55,3434.5,"[[13.873725, 45.709296], [13.873744, 45.709304..."
341,SL2,UD_A,117.72,4550.5,"[[13.873725, 45.709296], [13.873744, 45.709304..."
342,SL2,LEF,96.19,5100.7,"[[13.873725, 45.709296], [13.873744, 45.709304..."


In [18]:
type(all_routes.geom)

pandas.core.series.Series

In [21]:
all_routes.geom[0][1]

[13.76746, 45.650831]

In [24]:
# access to waypoints of each route
nroutes:int = all_routes.shape[0]
nroutes:int = 3 #example
for r in range (nroutes):
    nwp = len(all_routes.geom[r])
    for wp in range (nwp):
        coords = all_routes.geom[r][wp]
        print(r,wp, coords)


0 0 [13.767103, 45.650613]
0 1 [13.76746, 45.650831]
0 2 [13.767543, 45.650881]
0 3 [13.767895, 45.651096]
0 4 [13.768403, 45.651387]
0 5 [13.768473, 45.651396]
0 6 [13.768544, 45.6514]
0 7 [13.768622, 45.6514]
0 8 [13.7687, 45.651391]
0 9 [13.768773, 45.651378]
0 10 [13.768881, 45.651327]
0 11 [13.768978, 45.651282]
0 12 [13.769352, 45.651077]
0 13 [13.769943, 45.650746]
0 14 [13.770107, 45.650652]
0 15 [13.770146, 45.650631]
0 16 [13.770207, 45.6506]
0 17 [13.77035, 45.650552]
0 18 [13.770648, 45.650473]
0 19 [13.771113, 45.650346]
0 20 [13.771196, 45.650323]
0 21 [13.771265, 45.650305]
0 22 [13.772004, 45.650108]
0 23 [13.77212, 45.650077]
0 24 [13.772238, 45.650045]
0 25 [13.772484, 45.649979]
0 26 [13.772567, 45.649957]
0 27 [13.773051, 45.649834]
0 28 [13.773325, 45.649779]
0 29 [13.773403, 45.649762]
0 30 [13.773849, 45.649669]
0 31 [13.773914, 45.649657]
0 32 [13.774018, 45.649639]
0 33 [13.774774, 45.649617]
0 34 [13.775488, 45.649611]
0 35 [13.776101, 45.649607]
0 36 [13.7762